In [9]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import time
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Bidirectional, LSTM, SimpleRNN,Input
from tensorflow.keras.optimizers import Adam


In [10]:
# Load the dataset
df = pd.read_csv('../../dataset/preliminary_dataset.csv')  

# Separating features and target
X = df[['Number of Lanes', 'Status of driver', 'Nature of environment', 'Velocity of vehicle', 'Distance between vehicles', 'Breaking capability']]
y = df['Collussion']

X_scaled = X

# Reshape input to be 3D [samples, timesteps, features] for LSTM
X_scaled = np.reshape(X_scaled, (X_scaled.shape[0], 1, X_scaled.shape[1]))

# Split into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)


In [11]:
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

(80000, 1, 6) (80000,) (20000, 1, 6) (20000,)


# Model

In [12]:
def build_deep_rnn_bi_lstm():
    model = Sequential()

    # Input Layer
    model.add(Input(shape=(X_train.shape[1], X_train.shape[2])))

    model.add(SimpleRNN(units=100, activation='relu', return_sequences=True))
    model.add(Bidirectional(LSTM(units=100, activation='relu', return_sequences=True)))
    model.add(Bidirectional(LSTM(units=100)))

    # Output Layer
    model.add(Dense(units=1, activation='sigmoid'))

    # Compile the Model
    model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['mse', 'mae'])


    return model


In [13]:

# Instantiate and train the model
model = build_deep_rnn_bi_lstm()
import time
batch_size = X_train.shape[0]  # Use full dataset per batch to match experimental setup
start_time=time.time()
history = model.fit(
    X_train, y_train,
    epochs=50,
    # batch_size=batch_size,
    steps_per_epoch=1,
    verbose=1
)
end_time = time.time()



Epoch 1/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 11s 11s/step - loss: 0.6942 - mae: 0.5005 - mse: 0.2505
Epoch 2/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step - loss: 0.6928 - mae: 0.4998 - mse: 0.2498
Epoch 3/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 9s 9s/step - loss: 0.6914 - mae: 0.4991 - mse: 0.2491
Epoch 4/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step - loss: 0.6901 - mae: 0.4983 - mse: 0.2485
Epoch 5/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step - loss: 0.6887 - mae: 0.4976 - mse: 0.2478
Epoch 6/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 8s 8s/step - loss: 0.6872 - mae: 0.4967 - mse: 0.2470
Epoch 7/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step - loss: 0.6856 - mae: 0.4957 - mse: 0.2462
Epoch 8/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step - loss: 0.6838 - mae: 0.4945 - mse: 0.2453
Epoch 9/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step - loss: 0.6818 - mae: 0.4932 - mse: 0.2444
Epoch 10/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step - loss: 0.6796 - mae: 0.4917 - mse: 0.2433
Epoch 11/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step - loss: 0.6771 - mae: 0.4900 - mse: 0.2421
Epoch 

In [14]:
# Calculate total training time
total_training_time = end_time - start_time  # in seconds
epochs = 50 # e.g., 50
steps_per_epoch = 1# e.g., 2000

# Total steps
total_steps = epochs * steps_per_epoch

# Calculate time per step
time_per_step = total_training_time / total_steps  # in seconds
time_per_step_ms = time_per_step * 1000  # Convert to milliseconds
#micro seconds
time_per_step_us = time_per_step_ms * 1000

# Calculate time per epoch
time_per_epoch = total_training_time / epochs

# Display results
print(f"Total Steps: {total_steps}")
print(f"Time per Step: {time_per_step:.6f} sec ({time_per_step_ms:.2f} ms)")
print(f"Time per Step: {time_per_step_us:.2f} µs")
print(f"Time per Epoch: {time_per_epoch:.2f} sec")


Total Steps: 50
Time per Step: 3.998404 sec (3998.40 ms)
Time per Step: 3998403.87 µs
Time per Epoch: 4.00 sec


In [15]:
from sklearn.metrics import mean_squared_error
import numpy as np
# Make predictions
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print(f'Mean Squared Error for y_pred (MSE): {mse:.16f}')

# Calculate RMSE
rmse = np.sqrt(mse)
print(f'Root Mean Squared Error (RMSE): {rmse:.16f}')


#ACCUARCY
y_pred = (y_pred > 0.5).astype(int)
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.16f}')


625/625 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step
Mean Squared Error for y_pred (MSE): 0.0836063697934151
Root Mean Squared Error (RMSE): 0.2891476608817977
Accuracy: 0.9144000000000000
